In [1]:
!pip install pmdarima
!pip install workalendar
!pip install prophet
!pip install -q --upgrade linear-tree

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 33.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 2.7 MB/s eta 0:00:00
  Created wheel for pymeeus: filename=PyMeeus-0.5.12-py3-none-any.whl size=731999 sha256=ab6978ef8cabd049a30ec204e71aed482aa36e1a9e379bf910ec32b8b004a876
  Stored in directory: /root/.cache/pip/wheels/d6/67/78/aa2e8d108639dd23a5e9e72a4fc88bb44f5541894382712f48
Successfully built pymeeus


In [4]:
import datetime
import json
import os
from joblib import Parallel, delayed
from time import sleep, time
import logging

import itertools
import holidays
import keras
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pmdarima as pm
import requests
import seaborn as sns
import statsmodels
import statsmodels.tsa.api as sm
import tensorflow as tf
import xgboost as xgb
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation, Dropout
from lineartree import LinearBoostRegressor
from matplotlib import rcParams  # Used to set default paremeters
from prophet import Prophet
from prophet.diagnostics import cross_validation
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from statsmodels.graphics.tsaplots import month_plot, plot_acf, plot_pacf, quarter_plot
from workalendar.europe import UnitedKingdom

# Figure default parameters
Define the default font and font sizes for all the plots

In [5]:
sns.set_style("whitegrid")

# Set Default Fonts
rcParams["font.family"] = "sans-serif"
rcParams["font.sans-serif"] = ["Arial", "Tahoma"]

# Set Default font sizes
small_size = 12
medium_size = 14
large_size = 16

# # Change the font size for individual elements
matplotlib.rc("font", size=small_size)  # controls default text sizes
matplotlib.rc("axes", titlesize=small_size)  # fontsize of the axes title
matplotlib.rc("axes", labelsize=medium_size)  # fontsize of the x and y labels
matplotlib.rc("xtick", labelsize=small_size)  # fontsize of the tick labels
matplotlib.rc("ytick", labelsize=small_size)  # fontsize of the tick labels
matplotlib.rc("legend", fontsize=small_size)  # legend fontsize
matplotlib.rc("axes", titlesize=large_size)  # title fontsize

# **Load data**

In [25]:
# Define the file path variable
file_path = '/content/historic_demand_2009_2024.csv'

# Load the CSV file, setting the first unnamed column as the index
df = pd.read_csv(file_path, index_col=0)

# Drop any additional unnamed columns if they appear
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

# Convert the "settlement date" column to datetime and format as "yyyy-mm-dd"
df['settlement_date'] = pd.to_datetime(df['settlement_date']).dt.strftime('%Y-%m-%d')

# EDA and Feature engineering
In this section, I will go through several steps to understand and transform the original dataset. The steps I will follow are:

Data understanding. Have a first look at the timeseries dataset
Data preparation. Deal with null values, drop uninformative features and remove outliers
Feature understanding. Understand the trend and different seasonalities in the data

**Data understanding**

The dataset includes a description of all the columns, but for this project I will only use three columns:

SETTLEMET_DATA: date in format dd/mm/yyyy
SETTLEMENT_PERIOD: half hourly period for the historic outtunr occurred
TSD (Transmission System Demand). Transmission System Demand is equal to the ND plus the additional generation required to meet station load, pump storage pumping and interconnector exports. Measured in MW.

TSD is target variable and the aim is to predict the future demand using different models

In [26]:
df.sample(n=7)

,settlement_date,settlement_period,nd,tsd,england_wales_demand,embedded_wind_generation,embedded_wind_capacity,embedded_solar_generation,embedded_solar_capacity,non_bm_stor,...,ifa2_flow,britned_flow,moyle_flow,east_west_flow,nemo_flow,nsl_flow,eleclink_flow,scottish_transfer,viking_flow,is_holiday
91502,2014-03-22,5,26906,28867,24421,1006,2524,0,4520,0,...,0,926,2,-144,0,NaN,NaN,NaN,NaN,0
37610,2011-02-23,27,46380,48380,42104,1407,1734,7,109,0,...,0,0,-401,0,0,NaN,NaN,NaN,NaN,0
134044,2016-08-24,21,32301,33232,29098,416,4353,4870,11100,0,...,0,1004,0,-423,0,NaN,NaN,NaN,NaN,0
252149,2023-05-21,8,17389,18803,15650,602,6538,0,15493,0,...,992,502,-452,-457,999,1296.0,999.0,-556.0,0.0,0
161570,2018-03-21,3,31162,33181,28541,809,5978,0,13052,0,...,0,999,297,0,0,NaN,NaN,NaN,NaN,0
210376,2020-12-31,41,36410,37018,33516,1019,6527,0,13080,0,...,-1,0,400,504,1015,0.0,0.0,NaN,NaN,0
88425,2014-01-17,8,28241,31023,25388,805,2524,0,3403,0,...,0,1000,-180,-478,0,NaN,NaN,NaN,NaN,0
